# Algorithm 14: Fourier Embedding (Boltz)

Sinusoidal embedding for continuous values like noise levels.

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/modules/encoders.py`
- **Class**: `FourierEmbedding`

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
def fourier_embedding(t, dim=256, max_period=10000):
    """
    Fourier (sinusoidal) embedding.
    
    Encodes continuous values into high-dimensional representations
    using sinusoidal functions at different frequencies.
    
    Args:
        t: Input value (scalar or array)
        dim: Embedding dimension
        max_period: Maximum period for lowest frequency
    
    Returns:
        Embedding [dim] or [batch, dim]
    """
    t = np.atleast_1d(t).astype(np.float32)
    half = dim // 2
    
    # Frequencies: geometric series from 1 to 1/max_period
    freqs = np.exp(-np.log(max_period) * np.arange(half) / half)
    
    # Arguments for sin/cos
    args = t[:, None] * freqs[None, :]
    
    # Sinusoidal embedding
    emb = np.concatenate([np.cos(args), np.sin(args)], axis=-1)
    
    return emb.squeeze()

In [ ]:
def noise_level_embedding(sigma, dim=256, sigma_data=16):
    """
    Embedding for noise level sigma.
    
    Uses log-scale for better representation of noise levels.
    """
    # Log-scale transformation
    c_noise = 0.25 * np.log(sigma + 1e-8)
    return fourier_embedding(c_noise, dim)

In [ ]:
# Test
print("Test: Fourier Embedding")
print("="*60)

# Single value
t = 0.5
emb = fourier_embedding(t, dim=64)
print(f"Single embedding: {emb.shape}")
print(f"Norm: {np.linalg.norm(emb):.4f}")

# Batch
t_batch = np.array([0.1, 0.5, 1.0, 5.0, 10.0])
emb_batch = fourier_embedding(t_batch, dim=64)
print(f"\nBatch embedding: {emb_batch.shape}")

# Noise level embedding
sigmas = np.array([0.1, 1.0, 10.0, 50.0])
noise_emb = np.array([noise_level_embedding(s, dim=64) for s in sigmas])
print(f"\nNoise embeddings: {noise_emb.shape}")

In [ ]:
# Similarity analysis
print("\nSimilarity Analysis")
print("="*60)

sigmas = [0.1, 0.5, 1.0, 5.0, 10.0, 50.0]
embeddings = np.array([noise_level_embedding(s, dim=128) for s in sigmas])

# Cosine similarity
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
sim = (embeddings @ embeddings.T) / (norms @ norms.T)

print("Cosine similarity between noise levels:")
for i, s1 in enumerate(sigmas):
    row = "  ".join([f"{sim[i,j]:.2f}" for j in range(len(sigmas))])
    print(f"  sigma={s1:5.1f}: {row}")

## Key Insights

1. **Continuous Encoding**: Encodes continuous values
2. **Multi-frequency**: Uses multiple frequencies for rich representation
3. **Log-scale**: Noise levels are log-transformed
4. **Smooth Interpolation**: Nearby values have similar embeddings